In [5]:
import patchwork_pieces
from random import shuffle

In [6]:
class Player(object):
    """Player class"""
    name = ''
    buttons = 5
    income = 0
    empty_spaces = 81
    location = 0
    
    def __init__(self, name):
        """Player creation
        name: name of the player
        user_type: AI or human
        """
        self.name = name
        
    def get_time_left(self):
        return 53 - self.location
        
    def get_status(self):
        """Prints the status of the player"""
        print( 'Player: {}'.format(self.name))
        print( '\tbuttons: {}'.format(self.buttons))
        print( '\tincome: {}'.format(self.income))
        print( '\tempty spaces: {}'.format(self.empty_spaces))
        print( '\tlocation: {}'.format(self.location))
        print( '\ttime left: {}'.format(self.get_time_left()))
        
    def get_sm_status(self):
        """Prints the status of the player"""
        print( '{}: b:{}, i:{}, e:{}, loc:{}'.format(self.name, self.buttons, self.income, self.empty_spaces, self.location))
        
    def add_piece(self, piece):
        """Adds a piece to the player's quilt and updates the states
        """
        self.income += piece.buttons
        self.buttons -= piece.cost_buttons
        self.location += piece.cost_time
        self.empty_spaces -= piece.squares
        
    def receive_income(self):
        """Gives the player his income"""
        self.buttons += self.income
        
    def choose_move(self, board):
        """Asks the player for a move"""
        return 0

    def get_score():
        """Returns the current score if the game ended now"""
        return self.empty_spaces * -2 + self.buttons
    
    def get_estimated_score():
        """Should estimeate a score based on time left, income remaining, and 1x1s with passing"""
        return self.get_score()
    
class HumanPlayer(Player):
    def __init__(self):
        Player.__init__(self, 'Human')
    
    def choose_move(self,board):
        board.print_market()
        return int( input() )

class JoelBot(Player):
    def __init__(self):
        Player.__init__(self, 'JoelBot')
    
    def choose_move(self,board):
        return 0

class Piece(object):
    name = ''
    squares = 0
    buttons = 0
    cost_buttons = 0
    cost_time = 0

    def __init__(self, name, squares, buttons, cost_buttons, cost_time):
        self.name = name
        self.squares = squares
        self.buttons = buttons
        self.cost_buttons = cost_buttons
        self.cost_time = cost_time

class Board(object):
    board = range(54)
    board_income = [4.5, 10.5, 16.5, 22.5, 28.5, 34.5,40.5, 46.5,52.5]
    board_patches = [25.5, 31.5, 37.5, 43.5, 49.5]
    market = []
    PATCH = Piece('patch',1,0,0,0)

    def __init__(self):
        self.add_all_pieces()
        
    def add_all_pieces(self):
        piece_list = patchwork_pieces.load_pieces()
        # shuffle(piece_list)
        for p in piece_list:
            self.add_piece_to_market(p)
            
    def remove_piece_from_market(self,piece):
        self.market.remove(piece)
        
    def print_market(self, n=3):
        for p in self.market[:n]:
            print( '{}, b:{}, s:{}'.format(p.name,p.buttons,p.squares))
            
class Game(object):
    
    def __init__(self, p1, p2):
        self.player_one = p1
        self.player_two = p2
        self.board = Board()
        
    def reset():
        self.player_one.reset()
        self.player_two.reset()
        self.board.reset()
    
    def game_over(self):
        return ( self.player_one.get_time_left() <= 0 and self.player_two.get_time_left() <= 0 ) 
    
    def is_legal(self, player, piece):
        # Check if the player can afford it
        if player.buttons >= piece.cost_buttons:
            return True
        else:
            return False
    
    def earns_income(self, player, piece):
        """check if player crossed income"""
        for i in self.board.board_income:
            if player.location - piece.cost_time < i < player.location:
                return True
        return False
        
    def earns_patch(self, player, piece):
        """check if player collects a patch"""
        for i in self.board.board_patches:
            if player.location - piece.cost_time < i < player.location:
                self.board.board_patches.remove(i)
                return True
        return False
    
    def make_move(self, player, piece):
        '''Checks to see if the player receives an income or a patch and then adds a piece to the player's quilt
        player: the player adding the piece
        piece: the piece to be added to the quilt
        
        returns nothing.
        '''
        print('{} adds the {} piece to his board.'.format(player.name, piece.name))
        player.add_piece(piece)
        self.board.remove_piece_from_market(piece)
        if self.earns_income(player, piece):
            player.receive_income()
        if self.earns_patch(player, piece):
            print( '{} is given a patch'.format(player.name) )
            player.add_piece( self.board.PATCH )
        return
    
    def run_game(self):
        active_player = self.player_one
        inactive_player = self.player_two
        
        while not self.game_over():
            while active_player.location <= inactive_player.location and not self.game_over():
                choice_ix = active_player.choose_move(self.board)
                piece = self.board.market[choice_ix]
                self.make_move(active_player, piece)
            print('STATUS:')
            active_player.get_sm_status()
            inactive_player.get_sm_status()
            print('\n')
            active_player, inactive_player = inactive_player, active_player

In [7]:
p1 = JoelBot()
p2 = HumanPlayer()
game = Game(p1,p2)

NameError: name 'Piece' is not defined

In [71]:
game.run_game()

0/3 cross, b:1, s:6
1/4 cross, b:1, s:7
5/4 t, b:2, s:5
0
Joel  adds the 0/3 cross piece to his board.
STATUS:
Joel : b:5, i:1, e:75, loc:3
Jacob: b:5, i:0, e:81, loc:0


Jacob adds the 1/4 cross piece to his board.
STATUS:
Jacob: b:4, i:1, e:74, loc:4
Joel : b:5, i:1, e:75, loc:3


5/4 t, b:2, s:5
5/5 t, b:2, s:5
7/2 t, b:2, s:6
1
Joel  adds the 5/5 t piece to his board.
STATUS:
Joel : b:3, i:3, e:70, loc:8
Jacob: b:4, i:1, e:74, loc:4


Jacob adds the 5/4 t piece to his board.
Jacob adds the 7/2 t piece to his board.
STATUS:
Jacob: b:-5, i:5, e:63, loc:10
Joel : b:3, i:3, e:70, loc:8


2/2 half-t, b:0, s:4
3/4 half-t, b:1, s:5
2/3 h, b:0, s:7


KeyboardInterrupt: 

In [15]:
l = [1,2,3,4]